<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Attention_Mechanisms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn

# Define the Attention class
class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()
        self.attention = nn.Linear(hidden_dim, 1, bias=False)  # Linear layer for attention scores

    def forward(self, hidden_states):
        scores = self.attention(hidden_states)  # Compute attention scores
        weights = torch.softmax(scores, dim=1)  # Apply softmax to get attention weights
        context = torch.sum(weights * hidden_states, dim=1)  # Compute context vector as weighted sum of hidden states
        return context, weights

# Example usage
attention_layer = Attention(hidden_dim=128)
hidden_states = torch.randn(32, 10, 128)  # Batch of 32, sequence length 10, hidden dimension 128
context_vector, attention_weights = attention_layer(hidden_states)

# Print the shape of the context vector
print("Context vector shape:", context_vector.shape)  # Expected shape: [32, 128]